# Data Creation Process

In [35]:
# import requirements
import pandas as pd
import numpy as np

## Sources

[NY County Level Test Data](https://health.data.ny.gov/Health/New-York-State-Statewide-COVID-19-Testing/xdss-u53e/data)

[County Level Economic Variables](https://raw.githubusercontent.com/JieYingWu/COVID-19_US_County-level_Summaries/master/data/counties.csv)

[NY Municipality Codes](https://data.ny.gov/Government-Finance/NY-Municipalities-and-County-FIPS-codes/79vr-2kdi)


In [36]:
ny_data = pd.read_csv("./raw_data/New_York_State_Statewide_COVID-19_Testing.csv")
county_codes = pd.read_csv("./raw_data/NY_Municipalities_and_County_FIPS_codes.csv")
county_data = pd.read_csv("https://raw.githubusercontent.com/JieYingWu/COVID-19_US_County-level_Summaries/master/data/counties.csv")

## Creating County Level Testing and Economic Data 

In [37]:
ny_data.head()

,Test Date,County,New Positives,Cumulative Number of Positives,Total Number of Tests Performed,Cumulative Number of Tests Performed
0,06/07/2020,Albany,17,1978,1194,32194
1,06/07/2020,Allegany,0,51,149,3117
2,06/07/2020,Bronx,72,46052,3718,222329
3,06/07/2020,Broome,4,624,745,22789
4,06/07/2020,Cattaraugus,1,95,245,7036


#### To avoid problems due to differences in how the county names are written, we want to use FIPS (county code) to identify counties. Since the NY site does not have the county codes, we will use a dataset that maps  county names to FIPS and join the two datasets.

In [38]:
#correcting a naming difference between the datasets
county_codes.replace("St Lawrence","St. Lawrence",inplace=True)

#st lawrence has the wrong county code so we're replacing it 
stlaw = county_codes[county_codes['County Name']=='St. Lawrence'].replace(36099, 36089)
county_codes = county_codes[county_codes['County Name']!='St. Lawrence'].append(stlaw)

In [39]:
county_code_df = county_codes.groupby(['County Name','County FIPS']).size().reset_index().drop(0,axis=1)

In [40]:
#joining ny data w codes data to get FIPS info
ny_data_fips = ny_data.set_index("County").join(county_code_df.set_index("County Name")).astype({'County FIPS': 'int32'})

#### Collecting a list of columns that have the relevant infection and socioeconomic data we want to explore

In [41]:
relevant_ses_columns = ["Median_Household_Income_2018",\
                        "transit_scores - population weighted averages aggregated from town/city level to county"]
relevant_infection_columns = ['Test Date','New Positives',\
                              'Cumulative Number of Positives',\
                              'Total Number of Tests Performed','Cumulative Number of Tests Performed',]

#### Joining testing data with socioeconomic data

In [42]:
#joining infection w socioeconomic
joined_df = county_data[county_data['State']=='NY'\
                               ].set_index('FIPS').join(ny_data_fips.set_index('County FIPS'))
#dropping null column
joined_df = joined_df[relevant_ses_columns\
                                      +relevant_infection_columns].drop(36000)

In [43]:
#doing some clean up
joined_df = joined_df.reset_index().rename(columns={'index':'FIPS'})
joined_df['Test Date'] = pd.to_datetime(joined_df['Test Date'])

In [44]:
joined_df.head()

,FIPS,Median_Household_Income_2018,transit_scores - population weighted averages aggregated from town/city level to county,Test Date,New Positives,Cumulative Number of Positives,Total Number of Tests Performed,Cumulative Number of Tests Performed
0,36001,64536.0,7.100041e+09,2020-06-07,17.0,1978.0,1194.0,32194.0
1,36001,64536.0,7.100041e+09,2020-06-06,8.0,1961.0,751.0,31000.0
2,36001,64536.0,7.100041e+09,2020-06-05,12.0,1953.0,1164.0,30249.0
3,36001,64536.0,7.100041e+09,2020-06-04,11.0,1941.0,1179.0,29085.0
4,36001,64536.0,7.100041e+09,2020-06-03,10.0,1930.0,817.0,27906.0


In [45]:
#renaming columns
joined_df = joined_df.rename(columns={'Density per square mile of land area - Population':'Population Density',\
                        'transit_scores - population weighted averages aggregated from town/city level to county':'Transit Scores',\
                       'Median_Household_Income_2018':'Median Income',
                       'Cumulative Number of Positives': 'Cumulative Positives',
                       'Total Number of Tests Performed': 'Total Tests',
                       'Cumulative Number of Tests Performed': 'Cumulative Tests'})

In [46]:
joined_df.to_csv("./data/ny_county_data.csv")